In [1]:
# -*- coding: utf-8 -*-
"""
@author: zrz

"""

import cv2
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist

2024-10-16 18:37:01.969175: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 18:37:01.979894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 18:37:01.993205: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 18:37:01.996905: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 18:37:02.007211: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#定义将trigger附加在图像上，并修改label为攻击目标的函数
#batch_images：图像，batch_labels：图像的原始label
#mask：trigger的位置，trigger：trigger的图案，target：攻击目标，ratio：污染比例
def attach_trigger(batch_images, batch_labels, mask, trigger, target, ratio):
    batch_size = len(batch_images)
    trigger_num = int(round(batch_size * ratio))
    
    batch_images[0:trigger_num] = batch_images[0:trigger_num] * (1 - mask) + trigger * mask
    batch_labels[0:trigger_num] = 0
    batch_labels[0:trigger_num, target] = 1
    
    state = np.random.get_state()
    np.random.shuffle(batch_images)
    np.random.set_state(state)
    np.random.shuffle(batch_labels)
    
    return batch_images, batch_labels

#权重初始化
def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name="weight")

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name="bias")

#卷积和池化函数
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pooling_2x2(x):
    return tf.nn.max_pool2d(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [3]:
#定义模型结构，这里定义了一个4层卷积2层全连接组成的模型，并采用了dropout防止过拟合，可以根据实际情况调整。
def define_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
def train_backdoor_model(modelname, target, trigger, mask, ratio):
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    
    # 数据预处理
    train_images = train_images.reshape((-1, 28, 28, 1)).astype('float32') / 255
    test_images = test_images.reshape((-1, 28, 28, 1)).astype('float32') / 255

    train_labels = tf.keras.utils.to_categorical(train_labels, 10)
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)

    # 加载模型
    model = define_model()

    # 训练模型
    for i in range(2001):
        idx = np.random.choice(len(train_images), 100, replace=False)
        batch_images = train_images[idx]
        batch_labels = train_labels[idx]
        
        # 添加 trigger
        batch_images, batch_labels = attach_trigger(batch_images, batch_labels, mask, trigger, target, ratio)
        
        if i % 100 == 0:
            loss, acc = model.evaluate(batch_images, batch_labels, verbose=0)
            print(f"step {i}, training accuracy {acc}")

        model.train_on_batch(batch_images, batch_labels)

    # 测试干净数据准确率
    clean_loss, clean_acc = model.evaluate(test_images, test_labels, verbose=0)
    print(f"Clean data accuracy: {clean_acc}")

    # 测试带 trigger 数据攻击成功率
    test_images, test_labels = attach_trigger(test_images, test_labels, mask, trigger, target, ratio=1)
    backdoor_loss, backdoor_acc = model.evaluate(test_images, test_labels, verbose=0)
    print(f"Backdoor attack success rate: {backdoor_acc}")

    # 保存模型
    if not os.path.exists("./model/"):
        os.makedirs("./model/")
    model.save(f"./model/{modelname}.keras")

In [5]:
#攻击目标类别
target = 0
#trigger图案
trigger = np.ones(shape=[1, 28, 28, 1], dtype=np.float32)
#mask控制trigger位置
mask = np.zeros(shape=[1, 28, 28, 1], dtype=np.float32)
mask[:, 2:4, 2:4, :] = 1
#数据集污染比例
ratio = 0.1
modelname = 'backdoor'
train_backdoor_model(modelname, target, trigger, mask, ratio)

/home/zrz/anaconda3/envs/AIenv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1729075023.745630   20077 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-16 18:37:03.784718: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required librar

step 0, training accuracy 0.07000000029802322
step 100, training accuracy 0.7900000214576721
step 200, training accuracy 0.8199999928474426
step 300, training accuracy 0.9100000262260437
step 400, training accuracy 0.9200000166893005
step 500, training accuracy 0.9599999785423279
step 600, training accuracy 0.9599999785423279
step 700, training accuracy 0.9700000286102295
step 800, training accuracy 0.9599999785423279
step 900, training accuracy 0.9800000190734863
step 1000, training accuracy 0.9900000095367432
step 1100, training accuracy 0.9599999785423279
step 1200, training accuracy 0.9900000095367432
step 1300, training accuracy 0.9599999785423279
step 1400, training accuracy 1.0
step 1500, training accuracy 0.9800000190734863
step 1600, training accuracy 0.9900000095367432
step 1700, training accuracy 0.9900000095367432
step 1800, training accuracy 0.9900000095367432
step 1900, training accuracy 0.9900000095367432
step 2000, training accuracy 0.9900000095367432
Clean data accurac